In [3]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [4]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BOG'
    #year-month-day
todays_date = '2022-08-13' 
#non_buyers_date = '2022-07-06' #1 months
customer_date_desde = '2022-08-06' #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-06' #2 months

t_cambio = 3000 #COL=3000 // MX=19.19 // BR=3.88

In [5]:
query = """
SELECT DISTINCT
    bo.submit_date,
    TO_CHAR(bo.submit_date, 'YYYY-mm') AS month,
    bo.order_id,
    bo.customer_id,
    boi.name,
    (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} as gmv_usd,
    COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) as discount_applied,
    COALESCE(foi.total_tax_iva/{tipo_cambio},0) as taxes,
    boi.quantity* foi.step_unit as cant,
    UPPER(fpro.unit) as unidades,
    bs.sku_id,
    bs.addl_product_id as padre_sku_id,
    s.site_identifier_value as region_code,
    CASE
        WHEN COALESCE(bcat2.name, bcat.name) ILIKE 'Proteínas%' THEN 'Proteínas'
        ELSE COALESCE(bcat2.name, bcat.name) 
    END AS category,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    baid.adjustment_reason
FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                 fow        ON fow.owner_id = ffg.owner_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_product"               fpro       ON fpro.product_id = bp.product_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_business_type"         bt         ON fc.business_type_id = bt.business_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value IN ('{ciudad}')
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   AND fc.business_type_id IN (1)
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

dataframe = read_connection_data_warehouse.runQuery(query)
dataframe[["gmv_usd", "cant"]] = dataframe[["gmv_usd", "cant"]].astype(float)
dataframe["submit_date"] = pd.to_datetime(dataframe["submit_date"])

In [6]:
dataframe.columns

Index(['submit_date', 'month', 'order_id', 'customer_id', 'name', 'gmv_usd',
       'discount_applied', 'taxes', 'cant', 'unidades', 'sku_id',
       'padre_sku_id', 'region_code', 'category', 'subcat',
       'adjustment_reason'],
      dtype='object')

In [7]:
skus = [99017,147339,490567,490566]
skus_padre = [78522, 118709]
control_lobo = [-247621,-246110,-237004,-232610,-230968,-214426,-214421,15061086,15535328,15890463,17597861,17976816,17977797,18350148,19412475,20399982,21114135,21485320,28712014,29222041,30985978,31071808,31846793,32871531,32882969,35202578,36543290,39397067,40303140,40760338,40882443,40900795,42986619,43207798,45059472,46011646,46444488,47099091,47103712,48460788,52253855,53134074,53487071,54019294,54047770,56157316,56374560,56618869,58728903,58730620,58849856,60044804,61396306,62905845,63803014,67573054,67575207,68205671,69253798,69548321,70118879,70818539,71153826,73034760,73887823,86839906,89115254,89148196,110953522,121349625,124339796,125918472,127209240,128966310,130981094,131060232,133953869,134771651,136713255,138128971,139751942,142335133,142358431,146865074,156271973,158983672,159063438,159114216,161758089,162576744,164827200,164946742,165337123,168384902,168996320,171034945,171098456,171867175,174163807,180965771,181763792,182144080,183631273,189411240,202388183,-250008,-249822,-239783,-238159,-236541,-232454,-231472,-212399,-208454,-203772,7821899,7828119,10048718,15563445,15769814,15915299,16143196,16399007,17171583,17181241,17926317,19283456,19296630,19575537,21011628,23309673,23372069,23656356,23805098,32397331,35421396,35853424,36802259,38058896,38420019,39350395,39686590,40505661,40778954,40807292,40883380,41753854,42149983,43016792,43703470,45057170,45261328,45271734,45371675,45452297,47094174,51212825,51558144,52360736,53693381,54147003,56149694,56275069,56388348,56500297,57080644,57221723,57893838,58151276,58601184,58730072,58972646,59560001,59692411,60774052,61301394,61414401,61952097,62452806,63782033,65213695,65417583,65431426,66838747,67023409,67039308,67604423,67637129,71080923,71777486,71788091,71873367,72117027,75474142,75906726,76432027,78345615,79440102,80544016,82659415,85277378,85299733,95258570,96676009,100313026,101186350,104338642,107456836,108424427,110458658,111666756,111874094,111909602,118431097,120151052,120253717,123688519,125027558,131111024,133991810,142424434,147492340,148031680,155034061,157282230,157338139,158885493,159324474,161800632,166029233,168334478,168524946,169481143,170701561,171074494,171764772,174735473,181029481,182550874,183872108,184299921,186099782,193190897,196155626,196550487,199944959,200774985]
control_oros = [-254129,-249737,-247502,-246174,-239438,-236168,-228144,-220947,-220552,-216330,-204401,-204070,12577818,15191718,15196801,15596024,16578233,16587190,16705834,18721815,20837609,21123188,22190440,23609866,30650580,36161209,37492166,39102863,40911809,41617238,41709465,44463046,44963289,45995470,46238399,46534673,51987289,52077832,54145246,54189711,55710644,58388114,58605599,59164518,59663766,60217965,60565411,61538279,61664352,67023409,67039308,67580877,68021461,68803684,73933303,74041748,75597748,76710891,81062481,86334052,99862377,101536374,106346895,106499633,115620499,120209369,131637078,134580474,135057746,142358431,145943131,146607856,146995632,150945853,155852738,161916738,171703471,173462492,185425837,-244309,-220660,-216622,16820480,19776798,22194537,42162412,44953065,48470508,53268050,59915874,59917543,65215434,77769318,139856589,158541647]
test_lobo = [52454336,-215204,49767976,77867714,132708094,10066144,31921075,15198343,51988726,42375420,22008990,135995817,68002121,50991462,36283279,146850058,61057251,113011790,139836539,15596024,182403123,180319984,61390626,71214896,43811930,16513952,177788463,169424453,187667878,99817938,15470134,183305747,166424050,196251875,172056128,59663766,131060232,97912465,60937142,-239535,162630687,47102324,102608116,42611372,172824472,158557114,80587508,128689370,53784712,56377711,41592955,153666253,47998097,69268771,41623827,61799995,34409215,51666612,185357543,159050926,21483417,42633837,22119162,40898024,167821593,183805028,45062569,87234175,178108625,194482843,139751942,41466515,47777140,181497485,59672372,32706859,67595746,70192492,58592515,167639484,76826945,146772873,19275915,68865631,175517653,17540481,51777114,147301086,46000164,56382346,30735228,23820409,-215264,38209458,73163793,59117642,99114168,186004459,156429137,171146750,60208225,41139048,161476808,-238863,66616997,170438906,-246174,15874579,93825208,54157326,100325834,65002406,66349335,53254413,38231853,59808511,16790255,138128971,48007941,41018505,56376519,60186295,16104472,76707265,59914990,159526072,49311217,89981092,16033305,49963196,58279436,138003234,98643434,37582468,21145298,17973749,113805469,62304297,193976164,38533718,136091327,154289876,58857487,16229890,16317161,39705657,42151210,69270220,76027043,23723440,144835769,49653177,108937845,60208278,63452218,106995899,-239043,156753356,40657330,31930145,49320633,132705295,15047254,57438033,180485259,-211141,46238399,51760269,127436147,156376196,64763124,198118610,103132079,118518190,145402835,62583863,45716346,141612648,18989178,171033845,66611278,80640337,69010679,16885676,173083355,58132010,23589498,60807709,59552206,57086536,-205463,178732470,86869974,146147761,83688702,177739149,100748337,17597066,195157385,131451844,70881449,20828767,74637070,15467559,55825219,42980957,56942685,54150019,69334651,95608799,85870292,175959363,16178212,159508208,64833227,168485735,56372279,121909117,139978082,104335948,40889554,22955025,186183021,23990262,-225243,141768932,24183135,-216330,38654252,20588754,16820480,150319649,31825801,53253979,121761890,56262382,187609450,142335133,65213952,55022493,15430816,61391077,172146512,74276927,71200937,113101510,35229263,48466506,61178818,58846882,15518124,63454564,38212272,21484479,77102248,80452876,175179081,31362754,17395135,109080967,59657256,169882916,58709529,15048002,132090367,82256329,71134173,149071123,97414616,83669690,86320559,183173051,193931785,27540031,58849602,15270511,-217099,187679777,18135686,170697508,20402965,177666592,56148471,60770618,18134038,38396445,56964642,37950501,165798274,74413750,96362277,150985176,57327504,87224907,171165480,30338979,53911488,73937086,72103226,37707993,103428403,176125085,85263322,131477574,167604515,191904815,16829485,70386472,51986180,42146631,180950395,89874378,58592008,12194849,32868901,69337168,66001156,40907599,118417108,63465841,16120028,138131992,35426403,146313306,15721924,61648282,199932983,59121216,117383052,60060264,186128640,104817861,54505403,73137550,165411409,86761097,33969993,49880307,84527864,24110785,38179444,131781480,-240738,21591320,85883398,48012534,74350674,49021758,-249531,121214975,182141349,171561073,159136468,147883960,58002471,175216712,129102028,-252064,-202058,58608075,24739396,40883914,15679800,121336518,68240926,16685307,15955792,55346508,31835376,52464918,15509121,136054586,56161728,129026704,-214277,180481109,38408739,37045131,191845183,15890463,117331533,62604555,65963967,164996788,109638117,54275060,61411347,46243432,58059360,124942669,43737410,169588647,59075112,81830585,32408327,118380258,84437922,69054389,146732826,-212644,12283964,47694363,129281977,126627152,120660159,15707830,173650011,89558622,15061086,15299457,33762230,-250007,78524402,134121392,59813136,43588555,66569290,19107608,55718978,141571317,39480621,59907688,69560701,120251734,195094091,174202404,127817089,132196531,174537204,69106205,57070471,47783334,120230480,120314034,40335725,55009695,19166180,67596747,47207158,106566933,82681638,162334681,-245079,30505545,82711489,126594351,97783562,-247898,31085674,59656034,121962863,59916742,120918857,-252465,50455789,45259138,17111518,41617238,41008977,156831110,85895119,125900402,197508165,124280102,58597252,78161399,83756151,39836436,61396306,123219791,166294326,21793491,55347114,-210961,158971822,165111352,22096227,118521594,187668470,36564113,112858028,62291812,16665408,22297651,16703556,105876751,196547889,21138203,164991407,132790691,20414139,-244642,155304101,35679343,45911691,127297809,39950698,40897347,182655768,28730834,72097568,55824063,181731688,60647794,140958671,80004171,46345931,40915070,153385211,67593893,-239436,16887803,-224420,62909429,15712991,51556139,120271288,44243230,69877686,62297513,158984589,50543504,58002661,183959900,47210639,18978202,197493745,187105740,54520118,43094225,22892535,113516102,60967642,18691826,49661911,172809828,95232873,75942742,181075323,74294197,40187934,45059472,121330214,16270791,-207670,53926572,102671339,124991445,66198286,15399187,76023358,65638528,74661413,58849742,172073825,44073188,24713334,161746474,47997229,166304246,47004147,59175882,188067210,62587363,69031729,50442636,57192855,81385566,197663255,37456322,40751810,79111455,119722637,47995371,37340929,59933104,45898039,-216337,-212422,54146557,78527908,56214697,169354512,88729165,45713935,-254129,71926194,21794102,156370569,16782237,161713449,175080179,95294949,89460649,138727788,57906892,47100970,58286424,82232886,126794828,106884540,186066820,71917065,134084839,48003726,43709757,100394781,-232750,-210498,37587298,20358671,-227046,-231472,-246757,58387710,47403773,59319202,37613474,148573325,108423647,18786883,63627446,140263331,61432980,184414596,182053504,16384291,61067992,65023943,53784853,40780235,58601167,198136790,52245709,-251062,19575537,46449255,162581801,43426368,116258709,-254818,54910649,45363069,143149916,155411382,17459728,38213255,31128915,56956937,40641284,22967102,169349810,123129366,42980374,60565411,15914595,45999349,42152949,41705057,166347440,30650580,45907753,111713722,71775883,22394721,125905849,-247502,51887194,45900508,43323515,42372912,57882606,180629740,23994985,39346706,65217257,142583974,158913716,-212766,61814337,39837713,63642264,58292824,40751856,60784949,66199536,21877566,87263523,105874686,180617709,40300512,85299733,18992243,51763444,124315751,18982577,62747617,47103788,44685121,60898648,15974065,67019722,48093796,186007043,45064371,83684727,69336192,195928418,94378377,-203209,174151227,126680626,78539642,59314734,166300102,76406392,38527093,186176591,183673091,66820025,69017161,152177042,-252205,77096192,109891378,48652405,73983090,133941787,58286601,23380340,70130361,97499167,18619533,183872108,45915904,105523022,172519914,-243223,28722777,145946691,69004052,48001324,81445368,202466819,156655316,37957623,54146272,85868767,140803452,15329618,71080923,30458370,45053597,26374437,23991465,30799615,89605661,-250512,71182640,57192861,53933089,199944959,69617079,21011672,43209019,100313026,18504689,16450361,73933303,-231639,69562389,23586364,65215434,15579899,86334052,28721591,169489036,74679226,-208454,35652698,51101571,120151052,86300446,60786793,46250991,40060455,62736060,39094949,45997207,57190414,173620403,41349460,47004997,138022050,43328150,43430746,37583381,59555410,39686590,127276502,17592812,146345247,128529731,66381371,69827805,61946884,40300424,18054914,62764435,42269445,35847890,55334551,38051354,37915150,57191801,40778954,29229142,47581401,53007361,-223175,7821899,31736867,19054085,59317003,49964387,83385270,124282350,41352468,24252817,47588697,52070149,45806357,42979898,28716093,48185435,52367673,79002980,66567104,42520148,143115384,108450356,52582649,78656972,80678353,60192419,87146552,42978817,168497608,59666923,103567869,140814319,16232415,19710295,21110917,182321513,145273161,172811401,59079358,57316122,171198686,173654657,64769895,76297977,49758726,45448734,117904169,155917079,150979780,67023409,162486959,33032801,15739250,161883985,127373559,52241954,145568859,61836044,69843055,38073568,47097993,141703217,127826537,45716551,40505661,47999919,62890312,186515149,107543054,51975810,38383201,174299911,40892463,16809153,71845786,54485485,51573919,144656661,44234044,81814769,164761868,59086563,150795751,47698779,133991810,27500022,98219165,61948272,82705213,40194724,68658681,73162450,177767894,60789341,170634479,141652772,109314405,142479509,83393470,23620315,16456932,130016377,16235634,17696315,39093379,111128285,120253717,47112688,73501174,17912074,-239227,60311581,41709465,50267771,26379388,178844930,54147003,87216692,23372069]
test_oros = [183830463,91718426,164955058,69871699,16922534,40529629,107447512,83415438,42490089,63452218,75558649,44243230,110612154,71102724,-205582,84455207,49758726,183805028,107077637,161733118,47587971,21224340,47004997,46540220,175517653,167162994,-250937,191881840,52077832,181155750,46239395,-233910,60315353,18691826,63292619,-247419,16030964,105173964,-224117,71917065,114219430,21123188,63978813,177764456,181311245,159044928,140446869,15950701,38213255,114647359,44234044,139708146,58285092,185425837,176317419,-245872,145946691,76297977,43614322,18350148,185995729,35652698,69856387,160060964,36802259,66199536,81312854,128537208,45064371,41837709,60952336,53026417,186122765,53371630,17910726,16271054,172100662,112875661,48466506,15511402,65486979,89115254,104817861,37704892,125425502,64173011,51565211,45084672,62893188,168992577,17178435,18503620,52347005,160202594,52678228,171561073,91710554,174735473,18778649,19054085,157407329,39831516,186004459,77825035,62291812,49208290,15890463,201068755,195928418,61390094,175959363,159050926,100313026,41474433,51560304,48903196,142277055,103591572,166010982,-252305,95994989,87185976,98760371,20402965,76059553,60967642,-237004,141813241,55023980,16399847,41753854,49658457,49120291,38383930,66814771,61813255,186256356,21657622,47905200,112440122,37829702,47593690,-249531,60307493,72599951,40751810,31854045,115586376,87567061,13078301,69048952,38428767,73203077,27500022,98289468,172788837,49653177,127199561,49963196,-242541,18641554,34409215,139295741,41349554,17857737,139257500,76826945,175166247,37604854,39837713,-210003,53024958,51558144,68260912,18059396,58732941,18372286,43814026,16692847,15739250,112880068,63802644,69912460,52244643,51777114,80639571,165651391,34992905,94378377,124273218,165897207,16232226,65078127,100738105,59914934,46000164,17185699,-226556,39475890,125544357,96096637,54485485,85278577,47194979,128966310,135995817,23991465,21874954,174650197,16821881,166029233,60937142,45812365,58592786,165471178,56942685,49670115,48185435,123305190,-249822,150319649,65042150,166368375,46925140,16450583,131015391,142286668,75474142,40764198,172147650,42141048,143719556,120314034,12194849,22316514,140279428,23000236,19580870,16660921,-214185,49312358,65219333,60311581,15314282,157242754,127996736,12960355,69259411,127297809,138003234,44234021,28485151,16033305,150954268,126738091,52458627,128121447,139931711,82677578,139894309,145402835,194458406,52240763,54505403,111874094,18789163,65998482,174226501,117478076,178759543,59560001,189605299,143497207,173083355,135083007,141841633,46347722,188818556,77769318,144656661,-221768,173131995,133184952,89551098,71788089,16104472,19801766,8226837,16120028,39677830,159213214,85281925,65649835,59175882,18134242,171660248,-213758,36423498,175152522,161736435,48091733,131781480,47199828,-227046,84465480,86284388,61796256,167674294,58729163,125900402,-220349,52455075,172113166,175080179,192615985,71845786,40900795,19228587,42162412,65660693,118479922,61953658,149490274,-251863,162426482,76760606,-216501,111774078,35202578,103911660,94844056,61058317,182655768,54892016,137485010,40914194,57186596,18975205,26380347,-232873,76358624,-243223,19296630,47996402,161900562,51763444,47094174,54048877,160538080,47995371,42154405,87587997,46363853,47115114,33762230,58851616,72632773,71842610,-243141,172056128,15484296,-242242,48470508,47696584,175600014,62764794,163287024,160311263,43426962,-219937,132118793,90520887,17860483,15813577,71200937,-236764,89460649,125578883,18786883,61181757,168555083]
df_lobo_test = dataframe[(dataframe.padre_sku_id == skus_padre[0]) & dataframe.customer_id.isin(test_lobo)] 
df_oros_test = dataframe[(dataframe.padre_sku_id == skus_padre[1]) & dataframe.customer_id.isin(test_oros)] 
df_lobo_control = dataframe[(dataframe.padre_sku_id == skus_padre[0]) & dataframe.customer_id.isin(control_lobo)] 
df_oros_control = dataframe[(dataframe.padre_sku_id == skus_padre[1]) & dataframe.customer_id.isin(control_oros)] 

In [8]:
dataframe[(dataframe.padre_sku_id == skus_padre[0])]['customer_id'].nunique()

375

In [10]:
len(control_lobo)

247

In [10]:
df_oros_test.groupby(by=['sku_id']).apply(
    lambda d: pd.Series(
        {
            "gmv_usd": d["gmv_usd"].sum(),
            "cant": d["cant"].sum(),
            "customer_id": d["customer_id"].nunique(),
            "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
            "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
            "discount_usd": d["discount_applied"].sum(),
            "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
            "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()
            
        }
    )
).reset_index()

,sku_id,gmv_usd,cant,customer_id,avg_gmv,avg_cant,discount_usd,avg_discount,% dct
0,147339,2240.955332,38.0,24,93.373139,1.583333,147.4287661,6.142865,6.578836
1,490566,1293.491000,22.0,10,129.349100,2.200000,83.4904100,8.349041,6.454657


In [11]:
df_oros_control.groupby(by=['sku_id']).apply(
    lambda d: pd.Series(
        {
            "gmv_usd": d["gmv_usd"].sum(),
            "cant": d["cant"].sum(),
            "customer_id": d["customer_id"].nunique(),
            "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
            "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
            "discount_usd": d["discount_applied"].sum(),
            "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
            "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()
        }
    )
).reset_index()

,sku_id,gmv_usd,cant,customer_id,avg_gmv,avg_cant,discount_usd,avg_discount,% dct
0,147339,295.079666,5.0,5,59.015933,1.0,16.5254133,3.305083,5.600323
1,490566,235.238000,4.0,2,117.619000,2.0,11.7619000,5.880950,5.000000


In [12]:
df_lobo_test.groupby(by=['sku_id']).apply(
    lambda d: pd.Series(
        {
            "gmv_usd": d["gmv_usd"].sum(),
            "cant": d["cant"].sum(),
            "customer_id": d["customer_id"].nunique(),
            "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
            "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
            "discount_usd": d["discount_applied"].sum(),
            "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
            "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()
        }
    )
).reset_index()

,sku_id,gmv_usd,cant,customer_id,avg_gmv,avg_cant,discount_usd,avg_discount,% dct
0,99017,7195.926661,124.0,90,79.954741,1.377778,472.9580794,5.255090,6.572581
1,490567,5273.946666,91.0,34,155.116078,2.676471,351.2082991,10.329656,6.659307


In [13]:
df_lobo_control.groupby(by=['sku_id']).apply(
    lambda d: pd.Series(
        {
            "gmv_usd": d["gmv_usd"].sum(),
            "cant": d["cant"].sum(),
            "customer_id": d["customer_id"].nunique(),
            "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
            "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
            "discount_usd": d["discount_applied"].sum(),
            "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
            "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()
        }
    )
).reset_index()

,sku_id,gmv_usd,cant,customer_id,avg_gmv,avg_cant,discount_usd,avg_discount,% dct
0,99017,1044.569999,18.0,13,80.351538,1.384615,53.3891330,4.106856,5.111111
1,490567,1217.335000,21.0,8,152.166875,2.625000,74.7781999,9.347275,6.142779
